## Dependenceis

In [ ]:
!pip install omegaconf
!pip install  pyntcloud
!pip install open3d
!pip install OpenCV
!pip install Plotly
!pip install psutil requests
!apt install aria2
!apt-get install -y orca
!pip install torch #should be already downloaded on google collab, including but limiting those above 
import tarfile
import os
import pyarrow.feather as feather
import numpy as np
import plotly.graph_objs as go
from ipywidgets import Button, VBox
from IPython.display import display, clear_output
from PIL import Image
import matplotlib.pyplot as pyplot
import pyarrow.feather as feather
import open3d as o3d
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import math
from torch.utils.data import Dataset, DataLoader


## Downloading Training and testing data

In [ ]:
# Create directories for train and test files if they don't exist
!mkdir -p /argotrain
#!mkdir -p /argotest

# Train 1
!aria2c -x 16 https://s3.amazonaws.com/argoverse/datasets/av2/tars/motion-forecasting/train.tar -d /argotrain -o train-000.tar

# Test 1
#!aria2c -x 16 https://s3.amazonaws.com/argoverse/datasets/av2/tars/sensor/test-000.tar -d /argotest -o test-000.tar


## deccompressing

In [ ]:
with tarfile.open('/argotrain/train-000.tar', 'r') as tar:
    tar.extractall('/argotrain')


In [ ]:
list_of_parquet_files = os.listdir("/argotrain/train")
#"/argotrain/train/0000b0f9-99f9-4a1f-a231-5be9e4c523f7/scenario_0000b0f9-99f9-4a1f-a231-5be9e4c523f7.parquet"
list_of_parquet_files_string=[]
for i in range(199908):
  list_of_parquet_files_string.append("/argotrain/train/"+list_of_parquet_files[i]+"/scenario_"+list_of_parquet_files[i]+".parquet")



## Setting up parameters 

In [ ]:

# Parameters
covered_time = 80      # Observed timesteps
future_time = 30       # Future timesteps
max_objects = 80       # Maximum number of objects per scenario, made through analysis on average count of objects per scneriar, mean was 55.5
bins = 120              # Number of bins for discretization, made arbitrarly, no reason behind it
batch_size = 16        # Batch size for training
num_epochs = 100        # Number of training epochs
learning_rate = 1e-4   # Learning rate for optimizer

# Pre-determined Global Bin Ranges (Set based on our prior analysis, check the ReadMe for more info at https://github.com/hashemJaber/Motion-Predction-for-cars)
x_range_min = 0.0
x_range_max = 41.7139
y_range_min = -5.5628
y_range_max = 6.2710
heading_range_min = -0.7035
heading_range_max = 0.7346
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Preparing Data loaders

In [ ]:

# 1. Extract World Trajectories
def extract_world_trajectories(parquet_path:str, max_objects:int=80, observed_timesteps:int=80):
    """
    Extract world trajectories for the observed timesteps.
    Args:
        parquet_path: str  (Path to the .parquet file)
        max_objects: int (Maximum number of objects to extract)
        observed_timesteps: int (The number of timesteps to include as input (i.e., the history up to t))
    Returns:
        World trajectory array of shape (max_objects, observed_timesteps, feature_dim).
    """
    df = pd.read_parquet(parquet_path)
    df = df.sort_values(by=['track_id', 'timestep'])
    features = ['position_x', 'position_y', 'heading', 'velocity_x', 'velocity_y', 'object_category']

    grouped = df.groupby('track_id')
    objects = [object_group[features].to_numpy()[:observed_timesteps]
               for _, object_group in grouped if len(object_group) >= observed_timesteps]


    # Pad or truncate to max_objects, maybe a bad opition subject to change
    num_objects = min(len(objects), max_objects)
    objects_array = np.zeros((max_objects, observed_timesteps, len(features)))
    objects_array[:num_objects] = objects[:num_objects]
    #print("objects_array: ",objects_array.shape)
    #print("extract_world_trajectories: ",objects_array)

    return objects_array



# 2. Extract Focused Object Trajectory
def extract_focused_object_trajectory(parquet_path:str, observed_timesteps:int=80):
    """
    Extract the focused object's trajectory up to the observed timesteps.

    NOTE: THIS MIGHT SEEM REDUNTANT FOR NOW, DUE TO THE WORLD TRAJECTORIES ALREADY INCLUDING THIS INFO
    BUT THIS IS SUBJECT TO CHANGE ONCE WE RESOLVE THE HD MAP REPRESENATION

    Args:
        parquet_path: str (Path to the .parquet file)
        observed_timesteps: int (The number of timesteps to include as input (i.e., the history up to t))

    Returns:
        Focused object trajectory array of shape (observed_timesteps, feature_dim).
    """
    df = pd.read_parquet(parquet_path)
    focal_track_id = df['focal_track_id'].iloc[0]
    focused_object_df = df[df['track_id'] == focal_track_id]

    features = ['position_x', 'position_y', 'heading', 'velocity_x', 'velocity_y', 'object_category']
    focused_object_df=focused_object_df[features].to_numpy()[:observed_timesteps]
    #print("focused_object_df: ",focused_object_df.shape)

    # Pad or truncate to max_objects
    #num_objects = min(len(objects), max_objects)
    #print("focused_object_df: ",focused_object_df.shape)
    #print("focused_object_df: ",focused_object_df[:1])
    return focused_object_df

## Loading the model

In [5]:
import torch


checkpoint_path = '/Users/hashemjaber/Desktop/Motion-Predction-for-cars/best_model.pth' #replace this with your own path !
checkpoint = torch.load(checkpoint_path,  map_location=torch.device('cpu'))

checkpoint_contents = {
    "keys": list(checkpoint.keys()),
    "model_state_dict_keys": list(checkpoint['model_state_dict'].keys()) if 'model_state_dict' in checkpoint else "Not Available",
    "optimizer_state_dict_keys": list(checkpoint['optimizer_state_dict'].keys()) if 'optimizer_state_dict' in checkpoint else "Not Available",
    "epoch": checkpoint['epoch'] if 'epoch' in checkpoint else "Not Available",
    "loss": checkpoint['loss'] if 'loss' in checkpoint else "Not Available",
}

checkpoint_contents


/var/folders/74/0x8rjbtd3xb99tjy9f_2dth80000gn/T/ipykernel_94752/124414281.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path,  map_

{'keys': ['model_state_dict', 'optimizer_state_dict', 'epoch', 'loss'],
 'model_state_dict_keys': ['src_tok_emb.weight',
  'src_tok_emb.bias',
  'tgt_tok_emb.weight',
  'tgt_tok_emb.bias',
  'positional_encoding.pos_embedding',
  'transformer.encoder.layers.0.self_attn.in_proj_weight',
  'transformer.encoder.layers.0.self_attn.in_proj_bias',
  'transformer.encoder.layers.0.self_attn.out_proj.weight',
  'transformer.encoder.layers.0.self_attn.out_proj.bias',
  'transformer.encoder.layers.0.linear1.weight',
  'transformer.encoder.layers.0.linear1.bias',
  'transformer.encoder.layers.0.linear2.weight',
  'transformer.encoder.layers.0.linear2.bias',
  'transformer.encoder.layers.0.norm1.weight',
  'transformer.encoder.layers.0.norm1.bias',
  'transformer.encoder.layers.0.norm2.weight',
  'transformer.encoder.layers.0.norm2.bias',
  'transformer.encoder.norm.weight',
  'transformer.encoder.norm.bias',
  'transformer.decoder.layers.0.self_attn.in_proj_weight',
  'transformer.decoder.layers.0